In [1]:
import os
import numpy as np
import random
import cv2
import pandas as pd

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize
import dataloader as DT
from dataloader import get_dataloader

In [ ]:
'''
실행시키지 말 것 dataset zip파일 압축 푸는 코드
'''
# import zipfile
# DIV2K = zipfile.ZipFile('/home/lahj91/SR/DIV2K_valid_HR.zip')
# DIV2K.extractall('/home/lahj91/SR/DIV2K')
# DIV2K.close()

In [144]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./SR_training_datasets/BSDS200')#해당주소에서 data 이름 list로 추출
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("BSDS200_DataName.csv", index=False) #csv 파일로 저장

In [290]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./DIV2K/DIV2K_train_HR')
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("DIV2K_train_DataName.csv", index=False) #csv 파일로 저장

In [299]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./DIV2K/DIV2K_valid_HR')
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("DIV2K_valid_DataName.csv", index=False) #csv 파일로 저장

In [192]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('/home/lahj91/SR/SR_testing_datasets/BSDS100')
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("valid_BSDS200.csv", index=False) #csv 파일로 저장

In [5]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
name = os.listdir('/home/lahj91/SR/SR_training_datasets/General100')

df = pd.DataFrame("general100", columns="data")
df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
df.to_csv("train_General100.csv", index=False) #csv 파일로 저장

ValueError: DataFrame constructor not properly called!

In [7]:
'''
부가적인 함수들
'''
def img2tensor(img, normalize=True):
    # handle gray image
    if len(img.shape)==2:
        img = img[:,:,np.newaxis]
    img = np.transpose(img, axes=(2,0,1)).astype(np.float32)
    if normalize:
        img = img / img.max()
    return torch.from_numpy(img)

def tensor2img(tensor):
    if len(tensor.shape)==2:
        img = tensor.cpu().numpy()
    elif len(tensor.shape)==3:
        img = np.transpose(tensor.cpu().numpy(), axes=(1,2,0))
    return img

def read_img_as_tensor(img_path, img_size=224, normalize=True):
    if isinstance(img_size, int):
        img_size = (img_size, img_size)
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    img = img2tensor(img, normalize=normalize)
    return img

In [1]:
import os
import numpy as np
import random
import cv2
import pandas as pd

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize

from dataloader import get_dataloader


In [9]:
data_loader = get_dataloader(batch_size=16,
                                    setting='train', 
                                    augmentation=True,
                                    pin_memory=True,
                                    SR_mode = 2,
                                    data='DIV2K',
                                    data_merge=True,
                                    num_workers=0
                                    )

In [10]:
for items in data_loader:
    items = items
    break

In [ ]:
import dataloader as DT

In [9]:
exam = DT.dataset_SR()

In [12]:
exam.data

'DIV2K'

In [19]:
exam.__getitem__(2)['origin']

tensor([[[0.3749, 0.3749, 0.3642,  ..., 0.2571, 0.2571, 0.2714],
         [0.3678, 0.3607, 0.3678,  ..., 0.2678, 0.2642, 0.2714],
         [0.3607, 0.3571, 0.3678,  ..., 0.2535, 0.2607, 0.2678],
         ...,
         [0.2535, 0.2535, 0.2500,  ..., 0.2464, 0.2464, 0.2393],
         [0.2571, 0.2500, 0.2571,  ..., 0.2285, 0.2393, 0.2500],
         [0.2393, 0.2393, 0.2393,  ..., 0.2393, 0.2393, 0.2500]],

        [[0.4784, 0.4784, 0.4701,  ..., 0.3258, 0.3258, 0.3423],
         [0.4701, 0.4619, 0.4701,  ..., 0.3382, 0.3299, 0.3382],
         [0.4619, 0.4577, 0.4701,  ..., 0.3217, 0.3299, 0.3382],
         ...,
         [0.3588, 0.3588, 0.3546,  ..., 0.3052, 0.3052, 0.2969],
         [0.3629, 0.3546, 0.3629,  ..., 0.2845, 0.2969, 0.3093],
         [0.3423, 0.3423, 0.3423,  ..., 0.2969, 0.2969, 0.3093]],

        [[0.3153, 0.3153, 0.3087,  ..., 0.2693, 0.2693, 0.2825],
         [0.3087, 0.3022, 0.3087,  ..., 0.2792, 0.2726, 0.2792],
         [0.3055, 0.2989, 0.3120,  ..., 0.2660, 0.2726, 0.

In [12]:
data_name_list1 = pd.read_csv('./DataName/' + "train" + "_" + "DIV2K" + ".csv")
data_name_list2 = pd.read_csv('./DataName/' + "train" + "_" + "General100" + ".csv")
data_name_list3 = pd.read_csv('./DataName/' + "train" + "_" + "BSDS200" + ".csv")

In [17]:
list = [data_name_list1,data_name_list2,data_name_list3]
data_all=[]
for i in list:
    data_all.append(i)

In [18]:
data_all
data_all = pd.concat(data_all, axis=0, ignore_index=True)

In [19]:
data_all

,data,name
0,DIV2K,0706.png
1,DIV2K,0791.png
2,DIV2K,0027.png
3,DIV2K,0119.png
4,DIV2K,0481.png
...,...,...
1095,BSDS200,78019.png
1096,BSDS200,176019.png
1097,BSDS200,22013.png
1098,BSDS200,187039.png


In [10]:
data_name_list.iloc[0]['data']

'DIV2K'

In [7]:
import os
import numpy as np
import random
import cv2
import pandas as pd
import config
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize

class dataset_SR(Dataset):
    def __init__(self,
                setting = "train",
                augmentation = True,
                SR_mode = 2,
                data='DIV2K',
                data_merge = False
                ):
        super(dataset_SR, self).__init__()
        self.SR_mode = SR_mode
        self.data = data
        self.setting = setting
        # if self.setting == 'val':
        #     self.setting = 'test'
        self.augmentation = augmentation
        self.flip = 0.5
        self.rotation = [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE]
        self.degradation = [cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_AREA, cv2.INTER_CUBIC, cv2.INTER_LANCZOS4]
        self.channel_wise_noise = True
        self.MODE_PATH = config.MODE_PATH
        self.normalize_img = Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])
        self.prefix = self.MODE_PATH[self.setting]+ '/'+ self.data + '/'
        self.data_merge = data_merge

        if self.setting == "train":
            self.DATA_LIST = config.TRAINING_DATA_LIST
        elif self.setting == "test":
            self.DATA_LIST = config.TEST_DATA_LIST
        
        if self.data_merge:
            data_all = []
            for i in self.DATA_LIST:
                df = pd.read_csv('./DataName/' + self.setting + "_" + i + ".csv")
                data_all.append(df)
            self.data_name_list = pd.concat(data_all, axis=0, ignore_index=True)
        else:
            self.data_name_list = pd.read_csv('./DataName/' + self.setting + "_" + self.data + ".csv")

    def __len__(self):
        return len(self.data_name_list)

    def __getitem__(self, idx):
        if self.data_merge:
            view_path = self.MODE_PATH[self.setting]+ '/' + self.data_name_list.iloc[idx]['data'] + '/' + self.data_name_list.iloc[idx]['name']
        else:
            view_path = self.prefix + self.data_name_list.iloc[idx]['name']
        origin = cv2.imread(view_path, cv2.IMREAD_UNCHANGED).copy()
        p, q = random.randint(0, origin.shape[0]-self.SR_mode*48), random.randint(0, origin.shape[1]-self.SR_mode*48)
        origin = origin[p:p+self.SR_mode*48,  q:q+self.SR_mode*48] #random crop(96*96)으로 origin data 생성
        if self.augmentation:
            if self.channel_wise_noise: #channel-wise noise
                pn = np.random.uniform(*[1 - 0.4, 1 + 0.4], size=(3))
                origin = np.minimum(255., np.maximum(0., origin * pn[np.newaxis, np.newaxis, :]))
            if random.random() < self.flip: #좌우반전
                origin = cv2.flip(origin, 1)
            if self.rotation != False: #rotation
                rd = random.random()
                if rd < 3/4:
                    origin = cv2.rotate(origin, random.choice(self.rotation))
            degraded = cv2.resize(origin, dsize=(48,48), interpolation=random.choice(self.degradation)) 
        else:
            degraded = cv2.resize(origin, dsize=(48,48), interpolation=cv2.INTER_CUBIC)
        interpolated = cv2.resize(degraded, dsize=(self.SR_mode*48,self.SR_mode*48), interpolation=cv2.INTER_CUBIC) #bicubic으로 degraded를 96*96 size 복원 fx=0.5, fy=0.5,
        items = {"origin" : torch.from_numpy(origin.transpose(2,0,1))/255, "degraded" : self.normalize_img(torch.from_numpy(degraded.transpose(2,0,1))/255), "interpolated" : self.normalize_img(torch.from_numpy(interpolated.transpose(2,0,1))/255)}
        return items

def get_dataloader(batch_size=16, setting='train', augmentation=True, pin_memory=True, num_workers=0, **kwargs): #num_workers는 hyperparameter tunning의 영역
    if setting == 'train':
        augmentation = True
    elif setting == 'test':
        augmentation = False
    elif setting == 'valid':
        setting = 'test'
    dataloader = dataset_SR(setting=setting, augmentation=augmentation, **kwargs)
    return DataLoader(dataloader, batch_size=batch_size, shuffle=augmentation, pin_memory=pin_memory, num_workers=num_workers)

In [8]:
exam = dataset_SR(data_merge = True)

In [9]:
exam.__getitem__(1099)

{'origin': tensor([[[0.3341, 0.5214, 0.3745,  ..., 0.2056, 0.1946, 0.2020],
          [0.3709, 0.5214, 0.4259,  ..., 0.1983, 0.2056, 0.2020],
          [0.3452, 0.5214, 0.4810,  ..., 0.1946, 0.1946, 0.1946],
          ...,
          [0.2020, 0.4002, 0.4113,  ..., 0.1579, 0.1836, 0.1873],
          [0.2166, 0.4370, 0.4076,  ..., 0.1873, 0.1983, 0.1726],
          [0.2607, 0.4480, 0.4039,  ..., 0.2166, 0.2020, 0.1909]],
 
         [[0.2848, 0.3916, 0.3026,  ..., 0.3204, 0.3255, 0.3280],
          [0.3026, 0.3891, 0.3306,  ..., 0.3204, 0.3204, 0.3255],
          [0.2823, 0.3967, 0.3611,  ..., 0.3229, 0.3179, 0.3153],
          ...,
          [0.2289, 0.3331, 0.3407,  ..., 0.1780, 0.1958, 0.2009],
          [0.2390, 0.3535, 0.3382,  ..., 0.1983, 0.1983, 0.1907],
          [0.2619, 0.3636, 0.3382,  ..., 0.2187, 0.2085, 0.2009]],
 
         [[0.6430, 0.7832, 0.5560,  ..., 0.5560, 0.5560, 0.5560],
          [0.6866, 0.7929, 0.6092,  ..., 0.5512, 0.5463, 0.5512],
          [0.6382, 0.7929, 0.6